#### Байесовская классификация спам/не спам

Импортируем библиотеки

In [89]:
import pandas as pd
import sklearn.feature_extraction.text as tx, sklearn.naive_bayes as nb, sklearn.linear_model as lm, sklearn.ensemble as en, sklearn.metrics as me, sklearn.model_selection as ms
from warnings import filterwarnings as fw
fw('ignore')

Узнаем вариант

In [90]:
ord('Н')%3

0

Прочитаем корпус текстов

In [91]:
corpus = pd.read_csv('SMSpam.csv')
corpus

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


Проанализируем количество слов по классам:

In [92]:
cnt, cntS, cntH = 0, 0, 0
for i in corpus['Message']:
    cnt += len(i.split(' '))
for i in corpus[corpus['Category']=='spam']['Message']:
    cntS += len(i.split(' '))
for i in corpus[corpus['Category']=='ham']['Message']:
    cntH += len(i.split(' '))
print('Всего строк:', corpus.shape[0], '\nИз них', 
    corpus['Category'].value_counts()['spam'], 'spam и',corpus['Category'].value_counts()['ham'], 'ham', 
    '\nВсего слов: ', cnt, '\nиз них', cntS, 'spam и', cntH, 'ham')

Всего строк: 5572 
Из них 747 spam и 4825 ham 
Всего слов:  87265 
из них 17788 spam и 69477 ham


In [93]:
xtrain, xtest, ytrain, ytest = ms.train_test_split(corpus['Message'],corpus['Category'], test_size=0.2)
print('Обучающая выборка:', xtrain.shape,'\nТестовая выборка', xtest.shape)

Обучающая выборка: (4457,) 
Тестовая выборка (1115,)


-- -- 
Используем все слова с учетом регистра:

In [94]:
cv = tx.CountVectorizer(lowercase=False)
cv.fit(corpus['Message'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 10857 токенов


In [95]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97563   0.99792   0.98665       963
        spam    0.98462   0.84211   0.90780       152

    accuracy                        0.97668      1115
   macro avg    0.98012   0.92001   0.94723      1115
weighted avg    0.97686   0.97668   0.97590      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96976   0.99896   0.98414       963
        spam    0.99187   0.80263   0.88727       152

    accuracy                        0.97220      1115
   macro avg    0.98081   0.90080   0.93571      1115
weighted avg    0.97277   0.97220   0.97094      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99354   0.95846   0.97569       963
        spam    0.78495   0.96053   0.86391       152

    accuracy                        0.95874      1115
   macro avg    0.88924   0.95949   0.91980      1115
weighted avg 

-- -- 
Используем все слова, но в одном регистре:

In [96]:
cv = tx.CountVectorizer()
cv.fit(corpus['Message'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 8709 токенов


In [97]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98063   0.99896   0.98971       963
        spam    0.99254   0.87500   0.93007       152

    accuracy                        0.98206      1115
   macro avg    0.98658   0.93698   0.95989      1115
weighted avg    0.98225   0.98206   0.98158      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97172   0.99896   0.98515       963
        spam    0.99200   0.81579   0.89531       152

    accuracy                        0.97399      1115
   macro avg    0.98186   0.90738   0.94023      1115
weighted avg    0.97448   0.97399   0.97290      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99257   0.97092   0.98163       963
        spam    0.83815   0.95395   0.89231       152

    accuracy                        0.96861      1115
   macro avg    0.91536   0.96244   0.93697      1115
weighted avg 

-- -- 
Найдем все стоп-слова, которые встречаются в большом количестве сообщений (более 500 , что около 9% всей выборки) и уберем их:

In [98]:
cv = tx.CountVectorizer(max_df = 500)
cv.fit(corpus['Message'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов')
print(stop)

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 15 токенов
{'to', 'of', 'you', 'have', 'and', 'for', 'the', 'in', 'it', 'that', 'me', 'is', 'call', 'your', 'my'}


In [99]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97859   0.99688   0.98765       963
        spam    0.97761   0.86184   0.91608       152

    accuracy                        0.97848      1115
   macro avg    0.97810   0.92936   0.95187      1115
weighted avg    0.97846   0.97848   0.97790      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97169   0.99792   0.98463       963
        spam    0.98413   0.81579   0.89209       152

    accuracy                        0.97309      1115
   macro avg    0.97791   0.90686   0.93836      1115
weighted avg    0.97338   0.97309   0.97202      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99357   0.96262   0.97785       963
        spam    0.80220   0.96053   0.87425       152

    accuracy                        0.96233      1115
   macro avg    0.89788   0.96157   0.92605      1115
weighted avg 

Найдем все стоп-слова, которые встречаются в маленьком количестве сообщений (менее 10, что около 0.18% всей выборки), отфильтруем только те слова, которые не характерны какому-то классу (разница в частотах по классам не более, чем в 5 раз):

In [100]:
cv = tx.CountVectorizer(min_df = 10)
cv.fit(corpus['Message'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов отмечены как редкие.')
#print(stop) # не будем выводить список слов, так как он слишком большой

all_spam = corpus[corpus['Category']=='spam'].groupby('Category').agg({'Message':' '.join}).to_string()
all_ham = corpus[corpus['Category']=='ham'].groupby('Category').agg({'Message':' '.join}).to_string()
temp = set()
for i in stop:
    frH = all_ham.count(i + ' ')/cntH
    frS = all_spam.count(i + ' ')/cntS
    if(frS != 0 and frH != 0):
        if(not(frS/frH > 5 or frS/frH < 0.2)):
            temp.add(i)
stop = temp
print('Из них {0} удовлетворяют условиям.'.format(len(stop)))

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)
print('Работаем с {0} - {1} = {2}'.format(len(cv.vocabulary_) + len(stop), len(stop), len(cv.vocabulary_)), 'токенами.')

Всего 7691 токенов отмечены как редкие.
Из них 358 удовлетворяют условиям.
Работаем с 8709 - 358 = 8351 токенами.


In [101]:
cv = tx.CountVectorizer(stop_words = stop)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

In [102]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98163   0.99896   0.99022       963
        spam    0.99259   0.88158   0.93380       152

    accuracy                        0.98296      1115
   macro avg    0.98711   0.94027   0.96201      1115
weighted avg    0.98313   0.98296   0.98253      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97467   0.99896   0.98667       963
        spam    0.99219   0.83553   0.90714       152

    accuracy                        0.97668      1115
   macro avg    0.98343   0.91724   0.94690      1115
weighted avg    0.97706   0.97668   0.97583      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99258   0.97300   0.98270       963
        spam    0.84795   0.95395   0.89783       152

    accuracy                        0.97040      1115
   macro avg    0.92027   0.96347   0.94026      1115
weighted avg 

Используем встроенный список стоп-слов:

In [103]:
cv = tx.CountVectorizer(stop_words = 'english')
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97864   0.99896   0.98869       963
        spam    0.99242   0.86184   0.92254       152

    accuracy                        0.98027      1115
   macro avg    0.98553   0.93040   0.95561      1115
weighted avg    0.98052   0.98027   0.97968      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97270   0.99896   0.98566       963
        spam    0.99206   0.82237   0.89928       152

    accuracy                        0.97489      1115
   macro avg    0.98238   0.91066   0.94247      1115
weighted avg    0.97534   0.97489   0.97388      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99344   0.94289   0.96750       963
        spam    0.72637   0.96053   0.82720       152

    accuracy                        0.94529      1115
   macro avg    0.85990   0.95171   0.89735      1115
weighted avg 

-- -- 
Используем только слова с высокой важностью:

In [104]:
cv = tx.CountVectorizer()
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

e = en.RandomForestClassifier()
e.fit(tr_xtrain,ytrain)
e.score(tr_xtest,ytest)

0.9739910313901345

In [105]:
imp = pd.DataFrame({'name' : cv.vocabulary_.keys(), 'importance' : e.feature_importances_}) 
imp = imp.sort_values(by = 'importance', ascending = False)
top100 = imp.head(n=100)['name'].to_numpy()

cv = tx.CountVectorizer(vocabulary = top100)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.87545   1.00000   0.93359       963
        spam    1.00000   0.09868   0.17964       152

    accuracy                        0.87713      1115
   macro avg    0.93773   0.54934   0.55662      1115
weighted avg    0.89243   0.87713   0.83081      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.87785   1.00000   0.93495       963
        spam    1.00000   0.11842   0.21176       152

    accuracy                        0.87982      1115
   macro avg    0.93892   0.55921   0.57336      1115
weighted avg    0.89450   0.87982   0.83636      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.87882   0.98650   0.92955       963
        spam    0.61765   0.13816   0.22581       152

    accuracy                        0.87085      1115
   macro avg    0.74823   0.56233   0.57768      1115
weighted avg 

-- -- 
Используем tf-idf векторизатор слов в одном регистре:

In [106]:
tfidfv = tx.TfidfVectorizer()
tfidfv.fit(corpus['Message'])
tr_xtrain = tfidfv.transform(xtrain)
tr_xtest = tfidfv.transform(xtest)

In [107]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.95912   0.99896   0.97864       963
        spam    0.99107   0.73026   0.84091       152

    accuracy                        0.96233      1115
   macro avg    0.97510   0.86461   0.90977      1115
weighted avg    0.96348   0.96233   0.95986      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97368   0.99896   0.98616       963
        spam    0.99213   0.82895   0.90323       152

    accuracy                        0.97578      1115
   macro avg    0.98291   0.91395   0.94469      1115
weighted avg    0.97620   0.97578   0.97485      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98845   0.97715   0.98277       963
        spam    0.86503   0.92763   0.89524       152

    accuracy                        0.97040      1115
   macro avg    0.92674   0.95239   0.93900      1115
weighted avg 